## PC-wise full candidates votes lists over time

In [1]:
import pandas as pd
import os

In [2]:
pcfile = '2024-PC-attributes.csv'
folder1 = 'state_wise_results'
folder2 = 'pc_wise_results'

In [3]:
df1 = pd.read_csv(pcfile, dtype=str).fillna('')

In [4]:
df1.sample(5)

,OBJECTID,pc_id,s_u,st_code,st_name,pc_no,pc_name,pc_name_alias,status,State_LGD,...,Phase,DOP,Poll_Day,DOA,DOIN,LDFN,DSN,LDWC,DOCV,Date_of_Poll
306,336,818,S,20,RAJASTHAN,18,Jalore,Jalore,,8,...,2ND PHASE,,Friday,16-3-2024(Saturday),28-3-2024(Thursday),4-4-2024(Thursday),05-4-2024(Friday),08-4-2024(Monday,4-6-2024(Tuesday),4/26/2024
81,82,1040,S,4,BIHAR,40,Jamui (SC),Jamui,,10,...,1ST PHASE,,Friday,16-3-2024(Saturday),20-3-2024(Wednesday),28-3-2024(Thursday),30-3-2024(Saturday),2-4-2024(Tuesday),4-6-2024(Tuesday),4/19/2024
361,409,906,S,24,UTTAR PRADESH,6,Moradabad,Moradabad,,9,...,1ST PHASE,,Friday,16-3-2024(Saturday),20-3-2024(Wednesday),27-3-2024(Wednesday),28-3-2024(Thursday),30-3-2024(Saturday),4-6-2024(Tuesday),4/19/2024
241,215,2741,S,13,MAHARASHTRA,41,Latur,Latur,,27,...,3RD PHASE,,Tuesday,16-3-2024(Saturday),12-4-2024 (Friday),19-4-2024 (Friday),20-4-2024 (Saturday),22-4-2024(Monday),4-6-2024(Tuesday),5/7/2024
138,519,2915,S,10,KARNATAKA,15,Udupi Chikmagalur,Udupi Chikmagalur,,29,...,2ND PHASE,,Friday,16-3-2024(Saturday),28-3-2024(Thursday),4-4-2024(Thursday),05-4-2024(Friday),08-4-2024(Monday,4-6-2024(Tuesday),4/26/2024


In [5]:
pc_files = sorted(os.listdir(folder2)) # always sort!
pc_files[:2]

['pc_results_2024-06-04 16:30:28.csv', 'pc_results_2024-06-04 16:31:44.csv']

In [12]:
collector = []
missing_s_u_count = 0

for file1 in pc_files:
    df2 = pd.read_csv(os.path.join(folder2, file1),dtype=str).fillna('')
    if 'timestamp' not in df2.columns:
        print(f"Discarding {file1} as timestamp missing")
        continue
    
    if 'source_url' not in df2.columns:
        print(f"{file1} missing source_url, skiping")
        continue
    
    # clean up pc_no
    df2['pc_no'] = df2['pc_no'].apply(lambda x: x.replace('.0',''))
    
    # missing s_u : detect from source_url
    if 's_u' not in df2.columns:
        missing_s_u_count += 1
        df2['s_u'] = df2['source_url'].str.replace('https://results.eci.gov.in/PcResultGenJune2024/Constituencywise','').str[0]
    
    
    # add filename
    df2['source_file'] = file1
    
    # state_name and pc_name was wrongly recorded of UTs in some cases. need to replace from lookup
    if 'st_name' in df2.columns:
        df2.drop(columns=['st_name'], inplace=True)
    
    if 'pc_name' in df2.columns:
        df2.drop(columns=['pc_name'], inplace=True)
    
    collector.append(df2)

Discarding pc_results_2024-06-04 16:30:28.csv as timestamp missing


In [14]:
df3 = pd.concat(collector, ignore_index=True, sort=False).sort_values('timestamp').reset_index(drop=True)

In [20]:
# bringing in state and pc names
lookup_cols = ['s_u','st_code','st_name','pc_no','pc_name_alias']
df4 = pd.merge(df3, df1[lookup_cols], how='left', on=['s_u','st_code','pc_no']).rename(columns={'pc_name_alias':'pc_name'})
# rename 'pc_name_alias


In [21]:
df4.sample(10)

,st_code,pc_no,candidate_name,party,evm_votes,postal_votes,total_votes,vote_percentage,source_url,timestamp,s_u,source_file,st_name,pc_name
597278,10,17,DURGA PRASAD,Karunada Sevakara Party,2578,14,2592,0.18,https://results.eci.gov.in/PcResultGenJune2024...,2024-06-04 20:29:25,S,pc_results_2024-06-04 20:28:08.csv,KARNATAKA,Dakshina Kannada
1897829,11,18,P KRISHNAMMAL,Marxist Communist Party of India (United),537,12,549,0.06,https://results.eci.gov.in/PcResultGenJune2024...,2024-06-05 00:07:55,S,pc_results_2024-06-05 00:07:28.csv,KERALA,Kollam
3312267,22,37,A SEETHA,Bahujan Dravida Party,2476,-,2476,0.24,https://results.eci.gov.in/PcResultGenJune2024...,2024-06-05 04:19:33,S,pc_results_2024-06-05 04:18:47.csv,TAMIL NADU,Tenkasi
4198032,25,41,SHYAMALI PRADHAN,Communist Party of India (Marxist),98819,564,99383,6.5,https://results.eci.gov.in/PcResultGenJune2024...,2024-06-05 06:57:19,S,pc_results_2024-06-05 06:56:03.csv,WEST BENGAL,Bolpur
3454859,22,18,"PALANI, V.",Bahujan Samaj Party,6013,41,6054,0.53,https://results.eci.gov.in/PcResultGenJune2024...,2024-06-05 04:45:13,S,pc_results_2024-06-05 04:44:24.csv,TAMIL NADU,Tiruppur
2125349,24,70,GOPAL SINGH,Azad Adhikar Sena,729,8,737,0.06,https://results.eci.gov.in/PcResultGenJune2024...,2024-06-05 00:49:26,S,pc_results_2024-06-05 00:48:22.csv,UTTAR PRADESH,Ghosi
4128504,2,1,NABAM TUKI,Indian National Congress,100100,4579,104679,26.18,https://results.eci.gov.in/PcResultGenJune2024...,2024-06-05 06:45:25,S,pc_results_2024-06-05 06:45:22.csv,ARUNACHAL PRADESH,Arunachal West
1124537,4,6,ASHOK KUMAR YADAV,Bharatiya Janata Party,552705,723,553428,53.85,https://results.eci.gov.in/PcResultGenJune2024...,2024-06-04 22:10:30,S,pc_results_2024-06-04 22:10:19.csv,BIHAR,Madhubani
116988,6,15,UMESHBHAI NARANBHAI MAKWANA,Aam Aadmi Party,256902,4692,261594,24.98,https://results.eci.gov.in/PcResultGenJune2024...,2024-06-04 18:03:17,S,pc_results_2024-06-04 18:01:36.csv,GUJARAT,Bhavnagar
750112,22,7,A JAYACHANDRAN,Independent,534,-,534,0.05,https://results.eci.gov.in/PcResultGenJune2024...,2024-06-04 21:06:21,S,pc_results_2024-06-04 21:04:56.csv,TAMIL NADU,Arakkonam


In [22]:
len(df4)

4432191

In [23]:
df4.sample().T

,3781414
st_code,7
pc_no,4
candidate_name,ISHWAR JHAJHRIA
party,Independent
evm_votes,1121
postal_votes,1
total_votes,1122
vote_percentage,0.1
source_url,https://results.eci.gov.in/PcResultGenJune2024...
timestamp,2024-06-05 05:43:35


In [24]:
df4.columns

Index(['st_code', 'pc_no', 'candidate_name', 'party', 'evm_votes',
       'postal_votes', 'total_votes', 'vote_percentage', 'source_url',
       'timestamp', 's_u', 'source_file', 'st_name', 'pc_name'],
      dtype='object')

In [25]:
df5 = df4.drop_duplicates(['s_u', 'st_code', 'pc_no', 'candidate_name', 'evm_votes', 'postal_votes'])\
.sort_values(['s_u','st_code', 'pc_no', 'candidate_name', 'timestamp']).reset_index(drop=True)

In [26]:
len(df5)

113859

In [27]:
print(f"After de-duplicating entries where there were no changes recorded, data reduced from {len(df4)} to {len(df5)} rows")

After de-duplicating entries where there were no changes recorded, data reduced from 4432191 to 113859 rows


In [29]:
df5.sample(2)

,st_code,pc_no,candidate_name,party,evm_votes,postal_votes,total_votes,vote_percentage,source_url,timestamp,s_u,source_file,st_name,pc_name
16909,13,18,VIKAS CHHAGAN LAHANE,Independent,609,-,609,0.05,https://results.eci.gov.in/PcResultGenJune2024...,2024-06-04 20:19:13,S,pc_results_2024-06-04 20:17:39.csv,MAHARASHTRA,Jalna
62124,22,37,ESAI MATHIVANAN,Naam Tamilar Katchi,128685,1585,130270,12.54,https://results.eci.gov.in/PcResultGenJune2024...,2024-06-04 20:18:35,S,pc_results_2024-06-04 20:15:28.csv,TAMIL NADU,Tenkasi


In [30]:
columns_order = ['s_u', 'st_code', 'pc_no', 'st_name', 'pc_name', 'candidate_name', 'party',
       'evm_votes', 'postal_votes', 'total_votes', 'vote_percentage',
    'timestamp', 'source_url', 'source_file' ]

In [31]:
df5[columns_order].to_csv('Elections2024_candidates_votes_changes.csv', index=False)